In [1]:
# import stuff
import os
import numpy as np
import torch
import torch.utils.data as data
from itertools import product as product
import time

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import pandas as pd

# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

# set up person only VOC dataset

In [2]:
# load files
vocpath = os.path.join("..", "VOCdevkit", "VOC2007")
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath, cls="person")

# make Dataset
voc_classes = ['person']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 128  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(voc_classes)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase = "train", transform=transform, transform_anno = transform_anno)
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

batch_size = 64

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn, num_workers=8)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

000009
000017
000021
000023
000030
000032
000035
000041
000048
000050
000051
000066
000073
000081
000083
000089
000101
000104
000110
000113
000125
000129
000131
000133
000138
000146
000150
000159
000162
000163
000164
000165
000169
000170
000171
000173
000174
000177
000190
000192
000193
000194
000200
000210
000218
000220
000222
000229
000232
000245
000251
000257
000259
000269
000275
000276
000278
000282
000285
000288
000298
000302
000305
000308
000320
000321
000322
000323
000328
000331
000337
000338
000352
000359
000367
000372
000374
000382
000394
000406
000407
000411
000419
000428
000433
000435
000438
000443
000446
000448
000463
000468
000470
000476
000477
000480
000482
000483
000498
000499
000500
000515
000516
000518
000520
000523
000524
000525
000526
000530
000531
000535
000541
000545
000554
000555
000579
000583
000589
000591
000597
000612
000613
000625
000626
000628
000633
000648
000654
000677
000684
000688
000690
000694
000695
000702
000709
000717
000726
000731
000733
000739
000742

009496
009497
009499
009500
009504
009512
009517
009518
009520
009524
009526
009531
009532
009537
009541
009542
009546
009551
009557
009558
009565
009567
009568
009573
009577
009579
009585
009591
009596
009600
009603
009609
009611
009613
009614
009617
009618
009638
009641
009647
009649
009654
009655
009656
009659
009666
009668
009671
009676
009684
009687
009691
009693
009698
009703
009711
009712
009713
009717
009718
009721
009726
009732
009734
009738
009747
009754
009755
009756
009762
009767
009773
009776
009780
009781
009789
009792
009796
009800
009809
009813
009822
009828
009841
009845
009848
009851
009859
009867
009868
009869
009874
009877
009878
009879
009880
009881
009882
009900
009902
009917
009918
009926
009935
009942
009944
009946
009947
009949
009950
009954
009958


In [3]:
# check operation
batch_iterator = iter(dataloaders_dict["train"])  # iter
images, targets = next(batch_iterator)  # get first element
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].shape)  # check targets

torch.Size([64, 3, 128, 128])
64
torch.Size([1, 5])


In [4]:
targets[1]

tensor([[0.0000, 0.0000, 0.5851, 0.8360, 0.0000]])

# test with ssd model.

In [5]:
from utils.blazeface import SSD

BlazeFace(
  (features): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): BlazeBlock(
      (conv1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): ReLU(inplace)
    )
    (4): BlazeBlock(
      (conv1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, a

SSD(
  (blaze): BlazeFace(
    (features): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
          (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act): ReLU(inplace)
      )
      (4): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
   

In [6]:
# SSD300の設定
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': 128,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [16, 8],  # 各sourceの画像サイズ
    'steps': [8, 16],  # DBOXの大きさを決める
    'min_sizes': [16, 32],  # DBOXの大きさを決める
    'max_sizes': [32, 64],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

net = SSD(phase="train", cfg=ssd_cfg)

# SSDのweightsを設定

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# set inits for loc and conf
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

print("set weights!")

using: cuda:0
set weights!


In [7]:
print(net)

SSD(
  (blaze): BlazeFace(
    (features): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
          (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act): ReLU(inplace)
      )
      (4): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
   

In [8]:
from utils.ssd_model import MultiBoxLoss

# define loss
criterion = MultiBoxLoss(jaccard_thresh=0.5,neg_pos=3, device=device)

# optim
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

In [12]:
def get_current_lr(epoch):
    lr = 1e-3
    for i,lr_decay_epoch in enumerate([120,180]):
        if epoch >= lr_decay_epoch:
            lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    print("lr is:", lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("used device：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):
        
        adjust_learning_rate(optimizer, epoch)
        
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('train')
            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('val')
                else:
                    # 検証は5回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('Iteration {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/blazeface128_' +
                       str(epoch+1) + '.pth')

# start training here

In [14]:
num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

used device： cuda:0
lr is: 0.001
-------------
Epoch 1/200
-------------
train
Iteration 10 || Loss: 5.6041 || 10iter: 13.4629 sec.
Iteration 20 || Loss: 5.1559 || 10iter: 10.2217 sec.
Iteration 30 || Loss: 5.2245 || 10iter: 7.4033 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:178.0630 ||Epoch_VAL_Loss:0.0000
timer:  33.2429 sec.
lr is: 0.001
-------------
Epoch 2/200
-------------
train
Iteration 40 || Loss: 5.1779 || 10iter: 12.7278 sec.
Iteration 50 || Loss: 5.1793 || 10iter: 7.1467 sec.
Iteration 60 || Loss: 5.7415 || 10iter: 6.2813 sec.
-------------
epoch 2 || Epoch_TRAIN_Loss:178.7961 ||Epoch_VAL_Loss:0.0000
timer:  32.1239 sec.
lr is: 0.001
-------------
Epoch 3/200
-------------
train
Iteration 70 || Loss: 5.5289 || 10iter: 12.2706 sec.
Iteration 80 || Loss: 5.1408 || 10iter: 10.1684 sec.
Iteration 90 || Loss: 5.1711 || 10iter: 7.1539 sec.
-------------
epoch 3 || Epoch_TRAIN_Loss:179.8645 ||Epoch_VAL_Loss:0.0000
timer:  35.2760 sec.
lr is: 0.001
-------------
Epoch 4/200
---

Iteration 820 || Loss: 5.3574 || 10iter: 6.4120 sec.
-------------
val
-------------
epoch 25 || Epoch_TRAIN_Loss:180.6583 ||Epoch_VAL_Loss:91.2578
timer:  49.0977 sec.
lr is: 0.001
-------------
Epoch 26/200
-------------
train
Iteration 830 || Loss: 5.4789 || 10iter: 8.7715 sec.
Iteration 840 || Loss: 5.2433 || 10iter: 12.4867 sec.
Iteration 850 || Loss: 5.0040 || 10iter: 8.8925 sec.
-------------
epoch 26 || Epoch_TRAIN_Loss:177.9276 ||Epoch_VAL_Loss:0.0000
timer:  35.7462 sec.
lr is: 0.001
-------------
Epoch 27/200
-------------
train
Iteration 860 || Loss: 5.5082 || 10iter: 5.9661 sec.
Iteration 870 || Loss: 5.9313 || 10iter: 8.2595 sec.
Iteration 880 || Loss: 5.3088 || 10iter: 7.5498 sec.
Iteration 890 || Loss: 4.8905 || 10iter: 9.2868 sec.
-------------
epoch 27 || Epoch_TRAIN_Loss:179.4821 ||Epoch_VAL_Loss:0.0000
timer:  31.7952 sec.
lr is: 0.001
-------------
Epoch 28/200
-------------
train
Iteration 900 || Loss: 5.2306 || 10iter: 16.0498 sec.
Iteration 910 || Loss: 5.3216 |

-------------
epoch 49 || Epoch_TRAIN_Loss:177.8047 ||Epoch_VAL_Loss:0.0000
timer:  31.9851 sec.
lr is: 0.001
-------------
Epoch 50/200
-------------
train
Iteration 1620 || Loss: 5.3551 || 10iter: 9.9699 sec.
Iteration 1630 || Loss: 5.5118 || 10iter: 8.5303 sec.
Iteration 1640 || Loss: 5.7814 || 10iter: 6.4079 sec.
Iteration 1650 || Loss: 5.6199 || 10iter: 5.7744 sec.
-------------
val
-------------
epoch 50 || Epoch_TRAIN_Loss:178.7732 ||Epoch_VAL_Loss:91.1637
timer:  41.7314 sec.
lr is: 0.001
-------------
Epoch 51/200
-------------
train
Iteration 1660 || Loss: 5.9790 || 10iter: 21.3653 sec.
Iteration 1670 || Loss: 5.2513 || 10iter: 8.1523 sec.
Iteration 1680 || Loss: 5.0382 || 10iter: 6.1313 sec.
-------------
epoch 51 || Epoch_TRAIN_Loss:175.5281 ||Epoch_VAL_Loss:0.0000
timer:  37.7490 sec.
lr is: 0.001
-------------
Epoch 52/200
-------------
train
Iteration 1690 || Loss: 5.3843 || 10iter: 17.6204 sec.
Iteration 1700 || Loss: 4.9923 || 10iter: 11.5112 sec.
Iteration 1710 || Los

Iteration 2410 || Loss: 5.4569 || 10iter: 7.4716 sec.
Iteration 2420 || Loss: 5.4930 || 10iter: 15.9013 sec.
Iteration 2430 || Loss: 5.5266 || 10iter: 9.2243 sec.
Iteration 2440 || Loss: 5.7391 || 10iter: 6.4268 sec.
-------------
epoch 74 || Epoch_TRAIN_Loss:178.6583 ||Epoch_VAL_Loss:0.0000
timer:  40.8862 sec.
lr is: 0.001
-------------
Epoch 75/200
-------------
train
Iteration 2450 || Loss: 5.4786 || 10iter: 12.7070 sec.
Iteration 2460 || Loss: 5.4750 || 10iter: 6.9981 sec.
Iteration 2470 || Loss: 5.5104 || 10iter: 5.3650 sec.
-------------
val
-------------
epoch 75 || Epoch_TRAIN_Loss:179.1614 ||Epoch_VAL_Loss:91.0334
timer:  43.9434 sec.
lr is: 0.001
-------------
Epoch 76/200
-------------
train
Iteration 2480 || Loss: 5.4879 || 10iter: 9.3700 sec.
Iteration 2490 || Loss: 5.3558 || 10iter: 7.4044 sec.
Iteration 2500 || Loss: 6.1656 || 10iter: 3.6790 sec.
-------------
epoch 76 || Epoch_TRAIN_Loss:177.3414 ||Epoch_VAL_Loss:0.0000
timer:  23.0439 sec.
lr is: 0.001
-------------
E

Iteration 3220 || Loss: 5.1922 || 10iter: 5.2936 sec.
Iteration 3230 || Loss: 5.2714 || 10iter: 2.7261 sec.
-------------
epoch 98 || Epoch_TRAIN_Loss:176.8206 ||Epoch_VAL_Loss:0.0000
timer:  18.4160 sec.
lr is: 0.001
-------------
Epoch 99/200
-------------
train
Iteration 3240 || Loss: 5.6491 || 10iter: 7.6452 sec.
Iteration 3250 || Loss: 5.2761 || 10iter: 5.5755 sec.
Iteration 3260 || Loss: 5.5672 || 10iter: 3.2349 sec.
-------------
epoch 99 || Epoch_TRAIN_Loss:177.2025 ||Epoch_VAL_Loss:0.0000
timer:  18.5937 sec.
lr is: 0.001
-------------
Epoch 100/200
-------------
train
Iteration 3270 || Loss: 5.5807 || 10iter: 5.8068 sec.
Iteration 3280 || Loss: 5.6111 || 10iter: 5.9501 sec.
Iteration 3290 || Loss: 5.4797 || 10iter: 3.7781 sec.
Iteration 3300 || Loss: 5.2891 || 10iter: 2.6755 sec.
-------------
val
-------------
epoch 100 || Epoch_TRAIN_Loss:178.0156 ||Epoch_VAL_Loss:90.7770
timer:  23.0242 sec.
lr is: 0.001
-------------
Epoch 101/200
-------------
train
Iteration 3310 || Los

Iteration 4020 || Loss: 5.4939 || 10iter: 3.0155 sec.
-------------
epoch 122 || Epoch_TRAIN_Loss:176.9544 ||Epoch_VAL_Loss:0.0000
timer:  18.5220 sec.
lr is: 0.0001
-------------
Epoch 123/200
-------------
train
Iteration 4030 || Loss: 5.0363 || 10iter: 6.4393 sec.
Iteration 4040 || Loss: 5.5407 || 10iter: 5.7666 sec.
Iteration 4050 || Loss: 5.3622 || 10iter: 3.5525 sec.
-------------
epoch 123 || Epoch_TRAIN_Loss:176.0069 ||Epoch_VAL_Loss:0.0000
timer:  18.4261 sec.
lr is: 0.0001
-------------
Epoch 124/200
-------------
train
Iteration 4060 || Loss: 5.5153 || 10iter: 3.8196 sec.
Iteration 4070 || Loss: 5.5441 || 10iter: 7.1078 sec.
Iteration 4080 || Loss: 5.5459 || 10iter: 3.8972 sec.
Iteration 4090 || Loss: 5.5274 || 10iter: 2.6410 sec.
-------------
epoch 124 || Epoch_TRAIN_Loss:177.9515 ||Epoch_VAL_Loss:0.0000
timer:  18.2216 sec.
lr is: 0.0001
-------------
Epoch 125/200
-------------
train
Iteration 4100 || Loss: 5.4429 || 10iter: 8.6546 sec.
Iteration 4110 || Loss: 5.6398 || 

Iteration 4810 || Loss: 5.5521 || 10iter: 3.5166 sec.
-------------
epoch 146 || Epoch_TRAIN_Loss:177.3148 ||Epoch_VAL_Loss:0.0000
timer:  18.5824 sec.
lr is: 0.0001
-------------
Epoch 147/200
-------------
train
Iteration 4820 || Loss: 6.0957 || 10iter: 5.4086 sec.
Iteration 4830 || Loss: 5.5008 || 10iter: 5.6003 sec.
Iteration 4840 || Loss: 5.3099 || 10iter: 4.0849 sec.
Iteration 4850 || Loss: 5.2468 || 10iter: 2.7200 sec.
-------------
epoch 147 || Epoch_TRAIN_Loss:176.4210 ||Epoch_VAL_Loss:0.0000
timer:  18.3624 sec.
lr is: 0.0001
-------------
Epoch 148/200
-------------
train
Iteration 4860 || Loss: 5.3880 || 10iter: 9.8233 sec.
Iteration 4870 || Loss: 5.0581 || 10iter: 4.7309 sec.
Iteration 4880 || Loss: 5.8605 || 10iter: 2.6536 sec.
-------------
epoch 148 || Epoch_TRAIN_Loss:176.4308 ||Epoch_VAL_Loss:0.0000
timer:  18.5012 sec.
lr is: 0.0001
-------------
Epoch 149/200
-------------
train
Iteration 4890 || Loss: 5.8185 || 10iter: 8.2775 sec.
Iteration 4900 || Loss: 5.9017 || 

Iteration 5610 || Loss: 5.1039 || 10iter: 2.6859 sec.
-------------
val
-------------
epoch 170 || Epoch_TRAIN_Loss:176.2907 ||Epoch_VAL_Loss:90.4128
timer:  23.0759 sec.
lr is: 0.0001
-------------
Epoch 171/200
-------------
train
Iteration 5620 || Loss: 5.3225 || 10iter: 10.2095 sec.
Iteration 5630 || Loss: 5.5309 || 10iter: 4.5481 sec.
Iteration 5640 || Loss: 4.9989 || 10iter: 2.6340 sec.
-------------
epoch 171 || Epoch_TRAIN_Loss:179.5880 ||Epoch_VAL_Loss:0.0000
timer:  18.4328 sec.
lr is: 0.0001
-------------
Epoch 172/200
-------------
train
Iteration 5650 || Loss: 5.2309 || 10iter: 8.1992 sec.
Iteration 5660 || Loss: 5.4572 || 10iter: 5.3228 sec.
Iteration 5670 || Loss: 5.6440 || 10iter: 2.9831 sec.
-------------
epoch 172 || Epoch_TRAIN_Loss:174.2918 ||Epoch_VAL_Loss:0.0000
timer:  18.2854 sec.
lr is: 0.0001
-------------
Epoch 173/200
-------------
train
Iteration 5680 || Loss: 5.4925 || 10iter: 6.1745 sec.
Iteration 5690 || Loss: 5.2717 || 10iter: 5.9287 sec.
Iteration 5700

-------------
epoch 194 || Epoch_TRAIN_Loss:178.3666 ||Epoch_VAL_Loss:0.0000
timer:  18.3391 sec.
lr is: 1e-05
-------------
Epoch 195/200
-------------
train
Iteration 6410 || Loss: 5.5067 || 10iter: 8.7283 sec.
Iteration 6420 || Loss: 5.1139 || 10iter: 6.0315 sec.
Iteration 6430 || Loss: 4.9493 || 10iter: 2.7104 sec.
-------------
val
-------------
epoch 195 || Epoch_TRAIN_Loss:175.8441 ||Epoch_VAL_Loss:90.3797
timer:  23.4829 sec.
lr is: 1e-05
-------------
Epoch 196/200
-------------
train
Iteration 6440 || Loss: 5.4066 || 10iter: 6.9434 sec.
Iteration 6450 || Loss: 5.2994 || 10iter: 5.5812 sec.
Iteration 6460 || Loss: 5.5270 || 10iter: 3.2248 sec.
-------------
epoch 196 || Epoch_TRAIN_Loss:177.1734 ||Epoch_VAL_Loss:0.0000
timer:  18.1119 sec.
lr is: 1e-05
-------------
Epoch 197/200
-------------
train
Iteration 6470 || Loss: 5.2945 || 10iter: 4.9341 sec.
Iteration 6480 || Loss: 5.6017 || 10iter: 6.0553 sec.
Iteration 6490 || Loss: 5.4667 || 10iter: 3.8061 sec.
Iteration 6500 || 